モデルが全て同じものを返さないかをみる

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sys
import os

In [2]:
PATH = "../data/data/train_fish/"
# path devided into kinds of image
ALB = "ALB"
BET = "BET"
DOL = "DOL"
LAG = "LAG"
NoF = "NoF"
OTHER = "OTHER"
SHARK = "SHARK"
YFT = "YFT"
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
IMAGE_SIZE = 32
channnels=3

In [3]:

def accuracy(logits,labels):
    correct_prediction = tf.equal(tf.argmax(logits,1),tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
    return accuracy
def split_image_and_label(data_list,LESS=False):
    '''
    data_list should be [[data,label]...]
    '''
    data = []
    labels = []
    if LESS:
        i=0
        for value,label in data_list:
            i+=1
            data.append(value)
            labels.append(label)
            if i == 1000:
                break
        return data,labels
    else:
        for value,label in data_list:
            data.append(value)
            labels.append(label)
        return data, labels

In [4]:
# file load
def load_files(path):
    data = []
    save_pa = []
    for pa,_,_ in os.walk(path):
        if pa =='../data/data/train_fish/':
            continue
        save_pa.append(pa)
    for i_c, cat in enumerate(save_pa):
        for _,_,files in os.walk(cat):
            for file in files:
                label = np.zeros(8)
                label[i_c] = 1
                filepath = os.path.join(cat,file)
                data.append([filepath,label])
    return data
DATA = load_files(PATH)
import csv
with open("../data/meta.csv",'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    writer.writerow(DATA)

In [5]:
#データロード
# DATA = [(d[1],d[0]) for d in DATA]
import random
random.shuffle(DATA)
num_train = int(len(DATA) * 0.8)
train = DATA[:num_train]
test = DATA[num_train:]

#trainデータを三倍にかさ増しする。
train = train*3
test_image,test_label = split_image_and_label(test,False)


In [6]:
def flatten(image):
    image = image.flatten().astype(np.float32)/255.0
    return image
# opnecvによる読み込み
import cv2 
def convert_image_path_to_data_cv2(path,i):
    try:
#         img2 = cv2.imread(path,0) # 白黒画像で読み込み
        img2 = cv2.imread(path)

        img2 = cv2.resize(img2,dsize=(IMAGE_SIZE,IMAGE_SIZE),interpolation = cv2.INTER_CUBIC)
        np_image_data = np.asarray(img2)
        np_final = np.expand_dims(np_image_data,axis=0)
        np_final = flatten(np_final)
        if i % 1000 == 0:
            print(i)
        #1000 images / 15s
        return np_final
    except:
        print(path)
        print(i)
        return np.zeros([IMAGE_SIZE,IMAGE_SIZE])

In [7]:
# これらコードは、http://kivantium.hateblo.jp/entry/2015/11/18/233834から得られる
def inference(images_placeholder,keep_prob):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape,stddev=0.1)
        return tf.Variable(initial)
    def bias_variable(shape):
        initial = tf.constant(0.1,shape=shape)
        return tf.Variable(initial)
    def conv2d(x,W):
        return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')
    def max_pool_2x2(x):
        return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#     x_image = tf.reshape(images_placeholder,[-1,256,256,3])
    x_image = tf.reshape(images_placeholder,[-1,IMAGE_SIZE,IMAGE_SIZE,3])
    #  32 * 32 * 3 tensorへ変換する

    with tf.name_scope('conv1') as scope:
        W_conv1 = weight_variable([5,5,3,32])
        b_conv1 = bias_variable([32])
        h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) + b_conv1)

    with tf.name_scope('pool1') as scope:
        h_pool1 = max_pool_2x2(h_conv1)
        
    with tf.name_scope('conv2') as scope:
        W_conv2 = weight_variable([5,5,32,64])
        b_conv2 = bias_variable([64])
        h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)
        
    with tf.name_scope('pool2') as scope:
        h_pool2 = max_pool_2x2(h_conv2)
        # h_pool2 has (?, 64, 64, 64) shape
    #全結合
    with tf.name_scope('zenketsu1') as scope:
#         W_fc1 = weight_variable([64*64*64,1024])
        W_fc1 = weight_variable([64*64*1,1024])
        b_fc1 = bias_variable([1024])
#         h_pool2_flat = tf.reshape(h_pool2,[-1,64*64*64]) #3136 defined here
        h_pool2_flat = tf.reshape(h_pool2,[-1,64*64*1]) #3136 defined here
        
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)
        #dropout
        h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
        
    with tf.name_scope('zenketsu2') as scope:
        W_fc2 = weight_variable([1024,NUM_CLASSES])
        b_fc2 = bias_variable([NUM_CLASSES])
    
    with tf.name_scope('softmax') as scope:
        y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    
    return y_conv

In [8]:
! ls ../data/data/train_fish

ALB  BET  DOL  LAG  NoF  OTHER	SHARK  YFT


In [9]:
test_image,test_label = split_image_and_label(test,False) 
test_image = [convert_image_path_to_data_cv2(path,i) for i,path in enumerate(test_image)]

0


In [10]:
test_image[100]

array([0.10196079, 0.1254902 , 0.13725491, ..., 0.6313726 , 0.5647059 ,
       0.5372549 ], dtype=float32)

In [13]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        t_input = tf.placeholder(np.float32,name='input')
        saver = tf.train.import_meta_graph('../data/model/model.ckpt.meta',{'input':t_input})
        saved_path = '../data/model/model.ckpt'
        saver.restore(sess,saved_path)
        print("restored")
        graph = tf.get_default_graph()
        print(sess.run(tf.get_operations()'softmax_Softmax'))
        print(graph.get_operations())

        
#         print(sess.run(tf.get_collection(tf.GraphKeys.VARIABLES)))

#         print(sess.run([n.name for n in tf.get_default_graph().as_graph_def().node]))
#         print(sess.run(inference,feed_dict={
#             inputs: test_image
#         }))
        #test結果の答えと実際の値を見る
        # test_imageは実際にデータを取っている。
        for i,val in enumerate(test_image):
            result = inference(val,1.0)
            print(result)
            print(test_label[i])
            if i == 100:
                break

INFO:tensorflow:Restoring parameters from ../data/model/model.ckpt
restored


ValueError: Fetch argument 'softmax_Softmax' cannot be interpreted as a Tensor. ("The name 'softmax_Softmax' refers to an Operation not in the graph.")

In [ ]:
with tf.Graph().as_default():
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph('../data/model/model.ckpt.meta')
        saved_path = '../data/model/model.ckpt'
        saver.restore(sess,saved_path)
        print("restored")

        print(sess.run('gradients/softmax/MatMul_grad/tuple/control_dependency_1' , feed_dict={
            "Placeholder_1": test_image
        }))
#         print(sess.run(tf.get_collection(tf.GraphKeys.VARIABLES)))

#         print(sess.run([n.name for n in tf.get_default_graph().as_graph_def().node]))
#         print(sess.run(inference,feed_dict={
#             inputs: test_image
#         }))
        #test結果の答えと実際の値を見る
        # test_imageは実際にデータを取っている。
        for i,val in enumerate(test_image):
            result = inference(val,1.0)
            print(result)
            print(test_label[i])
            if i == 100:
                break